In [11]:
!pip install unidecode

In [12]:
import json
import re
from unidecode import unidecode
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
f = open('/content/drive/Shareddrives/MIT NLP 8.864/Data/drake.json')
drake = json.load(f)
f.close()

f = open('/content/drive/Shareddrives/MIT NLP 8.864/Data/tswift.json')
taylor = json.load(f)
f.close()

drake = [drake['songs'][i]['lyrics'] for i in range(len(drake['songs']))]
taylor = [taylor['songs'][i]['lyrics'] for i in range(len(taylor['songs']))]

taylor_lyrics = [re.sub('\u2005', ' ', re.sub(r'[\(\[].*?[\)\]]', '', taylor[i])).split('\n') for i in range(len(taylor))]
taylor_lyrics = [[unidecode(i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('\d+EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('\d+.EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[re.sub('EmbedShare URLCopyEmbedCopy', '', i) for i in taylor_lyrics[j]] for j in range(len(taylor_lyrics))]
taylor_lyrics = [[i for i in taylor_lyrics[j] if i != ''] for j in range(len(taylor_lyrics))]

drake_lyrics = [re.sub('\u2005', ' ', re.sub(r'[\(\[].*?[\)\]]', '', drake[i])).split('\n') for i in range(len(drake))]
drake_lyrics = [[unidecode(i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('\d+EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('\d+.EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[re.sub('EmbedShare URLCopyEmbedCopy', '', i) for i in drake_lyrics[j]] for j in range(len(drake_lyrics))]
drake_lyrics = [[i for i in drake_lyrics[j] if i != ''] for j in range(len(drake_lyrics))]

taylor_lyrics = [[line1 + ', ' + line2 for line1,line2 in zip(song[0::2], song[1::2])] for song in taylor_lyrics]
taylor_lyrics = [[line1 + ', ' + line2 for line1,line2 in zip(song[0::2], song[1::2])] for song in taylor_lyrics]

drake_tokenized = [[word_tokenize(drake_lyrics[i][j]) for j in range(len(drake_lyrics[i]))] for i in range(len(drake_lyrics))]
taylor_tokenized = [[word_tokenize(taylor_lyrics[i][j]) for j in range(len(taylor_lyrics[i]))] for i in range(len(taylor_lyrics))]

drake_tokenized = [[[word.lower() for word in line] for line in song] for song in drake_tokenized]
taylor_tokenized = [[[word.lower() for word in line] for line in song] for song in taylor_tokenized]

drake_length = sum([[len(sent) for sent in song] for song in drake_tokenized], [])
taylor_length = sum([[len(sent) for sent in song] for song in taylor_tokenized], [])

drake_lyrics = sum([[sent for sent in song if (len(sent) >= 10 and len (sent) <= 30)] for song in drake_tokenized], [])
taylor_lyrics = sum([[sent for sent in song if (len(sent) >= 10 and len (sent) <= 30)] for song in taylor_tokenized], [])

taylor_vocab = sum(taylor_lyrics,[])
drake_vocab = sum(drake_lyrics,[])

def unique(list1):
     
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list

vocab = taylor_vocab + drake_vocab
vocab_counts = Counter(vocab)
vocab = unique(vocab)
vocab = ['<pad>','<unk>','<s>', '</s>'] + vocab

from torch.utils import data
import torch

# These IDs are reserved.
MAX_SENT_LENGTH = 30
MAX_SENT_LENGTH_PLUS_SOS_EOS = 32
PAD_INDEX = 0
UNK_INDEX = 1
SOS_INDEX = 2
EOS_INDEX = 3
RARE_WORD_TRESHOLD = 0

vocab_counts['<pad>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['<unk>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['<s>'] = RARE_WORD_TRESHOLD + 1
vocab_counts['</s>'] = RARE_WORD_TRESHOLD + 1

class TSTDataset(data.Dataset):
    def __init__(self, taylor_sentences, drake_sentences, vocab, vocab_counts, sampling=1.):
        self.taylor_sentences = taylor_sentences[:int(len(taylor_sentences) * sampling)]
        self.drake_sentences = drake_sentences[:int(len(drake_sentences) * sampling)]

        self.max_seq_length = MAX_SENT_LENGTH_PLUS_SOS_EOS
        self.vocab = vocab
        self.vocab_counts = vocab_counts

        self.v2id = {v : i for i, v in enumerate(self.vocab)}
        self.id2v = {val : key for key, val in self.v2id.items()}
    
    def __len__(self):
        return min(len(self.taylor_sentences), len(self.drake_sentences))
    
    def __getitem__(self, index):
        taylor_sent = self.taylor_sentences[index]
        taylor_len = len(taylor_sent) + 2   # add <s> and </s> to each sentence
        taylor_id = []
        for w in taylor_sent:
            if w not in self.vocab:
                w = '<unk>'
            if vocab_counts[w] <= RARE_WORD_TRESHOLD:
                w = '<unk>'
            taylor_id.append(self.v2id[w])

        taylor_id = ([SOS_INDEX] + taylor_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_seq_length - taylor_len))

        drake_sent = self.drake_sentences[index]
        drake_len = len(drake_sent) + 2   # add <s> and </s> to each sentence
        drake_id = []
        for w in drake_sent:
            if w not in self.vocab:
                w = '<unk>'
            if vocab_counts[w] <= RARE_WORD_TRESHOLD:
                w = '<unk>'
            drake_id.append(self.v2id[w])

        drake_id = ([SOS_INDEX] + drake_id + [EOS_INDEX] + [PAD_INDEX] *
                  (self.max_seq_length - drake_len))

        return torch.tensor(taylor_id), taylor_len, torch.tensor(drake_id), drake_len

dataset = TSTDataset(taylor_lyrics, drake_lyrics, vocab, vocab_counts)
data_loader = data.DataLoader(dataset, batch_size=2, shuffle=True)

In [15]:
for a,b,c,d in data_loader:
    print(a)
    print([' '.join([dataset.id2v[word] for word in line]) for line in a.tolist()])
    print(c)
    print([' '.join([dataset.id2v[word] for word in line]) for line in c.tolist()])
    break

tensor([[   2, 6786,  833, 6132,  781, 2143, 7624, 2143, 6786, 6732,  781, 2143,
         6786, 6075, 6985,  808, 7185, 4841,  781,    3,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   2, 2215, 2143, 2215, 2143, 4988, 8342, 4839, 6001, 8020, 2143,  721,
         6075, 2594, 1971, 2059, 3137, 7393, 2143, 3036, 3397, 8642, 2594,  808,
         5580,    3,    0,    0,    0,    0,    0,    0]])
["<s> `` sparks fly '' , 2011 , `` superman '' , `` if this was a movie '' </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>", '<s> oh , oh , i got tired of waiting , wondering if you were ever coming around , my faith in you was fading </s> <pad> <pad> <pad> <pad> <pad> <pad>']
tensor([[   2, 7785, 2594, 4164, 6743, 5341, 3544, 2594, 3739, 2900, 2143, 4988,
         8342, 2594,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   2, 2594, 5483, 4234,

tensor([[   2, 4054, 6109, 3392, 3179, 7185, 3198, 2143, 4054, 6109, 3392, 3179,
         7185, 7666, 2143, 3179, 7185,  763, 7666, 2143, 7185, 8793, 6001, 4739,
         8281,    3,    0,    0,    0,    0,    0,    0],
        [   2, 5483, 4234, 2594, 4149, 6575, 3876, 1684, 2425, 2143,  528, 1744,
         4082, 2420, 3909, 2285, 2143, 2474, 1635, 2143, 7785, 4988, 8490, 2594,
          638, 6852, 2143, 4988, 7230, 8763,    3,    0]])
tensor([26, 31])
tensor([[   2, 8842, 8850, 2143, 8842, 7530, 2143, 6075, 2594, 4164, 8411, 8329,
         8218,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   2, 1823, 4560, 1118, 3955, 7185, 6914, 2143, 1422, 4234,  187, 1879,
            3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])
tensor([14, 13])
